### All the required imports are placed here

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Import the CIFAR10 dataset

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, 
                                              shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                          download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                            shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


# Neural Network Implementation

In [3]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        
        self.conv_part = nn.Sequential(
            nn.Conv2d(3, 96, 11, stride=4, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(3, stride=2),
            nn.BatchNorm2d(96),
            nn.Conv2d(96, 256, 5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(3, stride=2),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 384, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(3, stride=2)
        )
        
        self.connected_layers = nn.Sequential(
            nn.Linear(6 * 6 * 256, 4096),
            nn.ReLU(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, 10)
        )
        
    
    def forward(self, x):
        x = self.conv_part(x)
        x = x.view(-1, 6 * 6 * 256)
        x = nn.connected_layers(x)
        
        return x

In [29]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        
        return x

net = Net()

# Loss Function and Optimizer
- Using the cross entropy loss function
- Using Gradient Descent with Adam

In [30]:
learning_rate = 0.0001

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

# Training Loop

In [31]:
num_epochs = 20

for epoch in range(num_epochs):
    running_loss = 0.0
    
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        
        inputs, labels = Variable(inputs), Variable(labels)
        
        optimizer.zero_grad()
        
        #Forward Prop
        outputs = net(inputs)
        #Compute the loss for current batch
        loss = criterion(outputs, labels)
        #Back Prop
        loss.backward()
        #Gradient Descent
        optimizer.step()
        
        #print statistics
        running_loss += loss.data[0]
        if i % 100 == 99:
            print('[Epoch:{} {}] Loss: {}'.format(
                   epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0

print('Finished Training')

/home/abinav/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[Epoch:1 100] Loss: 0.11500381767749786
[Epoch:1 200] Loss: 0.11302738273143768
[Epoch:1 300] Loss: 0.11121015954017639
[Epoch:1 400] Loss: 0.10971905481815337
[Epoch:1 500] Loss: 0.10892418849468231
[Epoch:1 600] Loss: 0.10777596497535706
[Epoch:1 700] Loss: 0.10788285994529724
[Epoch:2 100] Loss: 0.10770377516746521
[Epoch:2 200] Loss: 0.10705357754230499
[Epoch:2 300] Loss: 0.10685428202152253
[Epoch:2 400] Loss: 0.1065369849205017
[Epoch:2 500] Loss: 0.10632976341247559
[Epoch:2 600] Loss: 0.1061022025346756
[Epoch:2 700] Loss: 0.10607893180847168
[Epoch:3 100] Loss: 0.10604389446973801
[Epoch:3 200] Loss: 0.1063132677078247
[Epoch:3 300] Loss: 0.10544590985774994
[Epoch:3 400] Loss: 0.10568157655000687
[Epoch:3 500] Loss: 0.10549018800258636
[Epoch:3 600] Loss: 0.10549830341339111
[Epoch:3 700] Loss: 0.10528148841857911
[Epoch:4 100] Loss: 0.10530574476718903
[Epoch:4 200] Loss: 0.10465124189853668
[Epoch:4 300] Loss: 0.10485536295175553
[Epoch:4 400] Loss: 0.1053181688785553
[Epo

Process Process-137:
Process Process-138:


KeyboardInterrupt: 

Traceback (most recent call last):
  File "/home/abinav/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/abinav/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/abinav/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
    r = index_queue.get()
  File "/home/abinav/anaconda3/lib/python3.6/multiprocessing/queues.py", line 342, in get
    res = self._reader.recv_bytes()
  File "/home/abinav/anaconda3/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/abinav/anaconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/abinav/anaconda3/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
Traceback (most recent call last

In [28]:
correct = 0
total = 0

for data in testloader:
    images, labels = data
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: {} %%'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 54.97 %%
